In [54]:
!pip install numpy
!pip install boto3
import numpy as np
import utils
import boto3
import pickle
import os

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


In [50]:
BASE_IMAGE = 'docker.io/pytorch/pytorch:1.0-cuda10.0-cudnn7-runtime'
bucket_name = 'elyra'
S3_END_POINT = "http://minio-kubeflow.apps.cluster-3f07.3f07.sandbox333.opentlc.com"
S3_ACCESS_ID = "minio"
S3_ACCESS_KEY = "minio123"

In [51]:
s3_endpoint_url = 'http://minio-kubeflow.apps.cluster-3f07.3f07.sandbox333.opentlc.com'
s3_access_key = "minio"
s3_secret_key = "minio123"

s3 = boto3.client(service_name='s3',
              	aws_access_key_id = s3_access_key,
              	aws_secret_access_key = s3_secret_key,
              	endpoint_url=s3_endpoint_url)

s3.list_buckets()['Buckets']
s3.create_bucket(Bucket=bucket_name)


{'ResponseMetadata': {'RequestId': '167E0F193D315A18',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'content-security-policy': 'block-all-mixed-content',
   'location': '/elyra',
   'server': 'MinIO/RELEASE.2019-08-14T20-37-41Z',
   'vary': 'Origin',
   'x-amz-request-id': '167E0F193D315A18',
   'x-xss-protection': '1; mode=block',
   'date': 'Tue, 11 May 2021 16:11:56 GMT',
   'set-cookie': 'c58b009a839de7a40fe40da5a53579fe=95ae0beae9ab3237ee669bd58d5e3350; path=/; HttpOnly'},
  'RetryAttempts': 0},
 'Location': '/elyra'}

In [52]:
def write_to_store(bucket, data, key):
    s3.put_object(Body=pickle.dumps(data),
                      Bucket=bucket,
                      Key=key)
    
def download_data() -> int:
    '''Download and store data in persistent storage
    '''

    def generate_binary_data(N_examples=1000, seed=None):
    #Generate N_examples points with two features each
    #
    #Args:
    #    seed: seed that should be fixed if want to generate same points again    
    #Returns:
    #    features: A 2-dimensional numpy array with one row per example and one column per feature
    #    target: A 1-dimensional numpy array with one row per example denoting the class - 0 or 1

        if seed is not None:
            np.random.seed(seed)

        features = []
        target = []

        for i in range(N_examples):
            #class = 0
            r = np.random.uniform() #class 0 has radius between 0 and 1
            theta = np.random.uniform(0, 2*np.pi) #class 0 has any angle between 0 and 360 degrees

            features.append([r*np.cos(theta), r*np.sin(theta)])
            target.append(0)

            #class = 1
            r = 3 + np.random.uniform() #class 1 has radius between 3+0=3 and 3+1=4
            theta = np.random.uniform(0, 2*np.pi) #class 1 has any angle between 0 and 360 degrees

            features.append([r*np.cos(theta), r*np.sin(theta)])
            target.append(1)

        features = np.array(features)
        target = np.array(target)

        return features, target

    features_train, target_train = generate_binary_data(N_examples=1000, seed=100)
    features_test, target_test = generate_binary_data(N_examples=500, seed=105)
    print("Writing features")
    write_to_store(bucket_name, features_train, 'features_train')
    write_to_store(bucket_name, target_train, 'target_train')
    write_to_store(bucket_name, features_test, 'features_test')
    write_to_store(bucket_name, target_test, 'target_test')
    print("Upload Files Complete")

    return 0

In [53]:
retcode = download_data()
if retcode > 0: raise ValueError("Step 1a failed")

Writing features
